<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

このノートブックは、**Default Spark Python** ランタイム環境を使用して、Watson Studio プロジェクトで実行する必要があります。**このノートブックを Watson Studio で表示していて、画面右上に Python 3.6 with Spark が表示されていない場合は、今すぐランタイムを更新してください。** 以下のクラウドサービスのサービス認証情報が必要です。
  * Watson OpenScale
  * Watson Machine Learning
 
  
有料のクラウドアカウントをお持ちの場合は、Watson Studio との統合や継続学習サービスを最大限に活用するために **Databases for PostgreSQL** または **Db2 Warehouse** サービスをプロビジョニングすることもできます。この有料サービスをプロビジョニングしないことを選択した場合、OpenScaleで無料の内部PostgreSQLストレージを使用できますが、モデルの継続学習を構成することはできません。

このノートブックでは、German Credit リスク　モデルをトレーニング、作成、展開し、その展開を監視するために OpenScale を設定し、7 日分の履歴記録と測定値を注入して OpenScale Insights ダッシュボードで表示します。

### Contents

- [セットアップ](#setup)
- [モデルの構築とデプロイ](#model)
- [OpenScaleの構成](#openscale)
- [モデル性能モニタリングとフィードバックロギング](#quality)
- [公平性、ドリフトモニタリング と 説明](#fairness)
- [カスタム・モニターと指標の作成](#custom)
- [ペイロード分析 ](#analytics)
- [ビジネス アプリケーション](#application)
- [履歴データ](#historical)
- [ビジネス アプリケーションの実行](#run_application)

# セットアップ <a name="setup"></a>

## Spark の確認

In [ ]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise 

## パッケージの導入

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client | tail -n 1
!pip install numpy --no-cache | tail -n 1
!pip install SciPy --no-cache | tail -n 1
!pip install lime --no-cache | tail -n 1

In [ ]:
!pip show ibm-ai-openscale

## サービスのプロビジョニングと資格情報の設定

もし、まだ IBM Watson OpenScale のインスタンスをプロビジョニングしていない場合は、[クラウド・カタログの OpenScale リンクを使って](https://cloud.ibm.com/catalog/services/watson-openscale)、IBM Watson OpenScale のインスタンスをプロビジョニングしてください。

Cloud API キーは、クラウド・コンソールの [**ユーザー** セクション](https://cloud.ibm.com/iam#/users) にアクセスして生成できます。そのページから、自分の名前をクリックして **APIキー** セクションまでスクロールし、**IBM Cloud API キーの作成** をクリックします。キーに名前を付けて **作成** をクリックし、作成したキーをコピーして以下に貼り付けます。

**注:**IBM CLOUD CLI(コマンンドラインインターフェース)を使ってOpenScaleの `API_KEY` を取得することもできます。

IBM CLOUD CLIのインストール方法: [インストラクション](https://cloud.ibm.com/docs/cli/reference/ibmcloud?topic=cloud-cli-install-ibmcloud-cli#install_use)

CLIを使ってAPIキーを取得する方法:

```
ibmcloud login
ibmcloud iam api-key-create 'my_key'
```

In [ ]:
CLOUD_API_KEY = "PASTE HERE"

In [ ]:
# The code was removed by Watson Studio for sharing.

次に、Watson Machine Learning(WML) の資格情報が必要です。すでに WML インスタンスを持っている場合は、そのインスタンスの資格情報を使用することができます。WML の新しい Lite インスタンスをプロビジョニングするには、[Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning) を使用し、サービス名を指定して、**作成** をクリックします。インスタンスが作成されたら、画面左側の**サービス資格情報**リンクをクリックします。**新規資格情報**ボタンをクリックし、資格情報に名前を付けて、**追加**をクリックします。新しいクレデンシャルは、**資格情報の表示**ボタンをクリックしてアクセスできます。WMLの資格情報をコピーして下のセルに貼り付けます。

In [ ]:
WML_CREDENTIALS = {
    "apikey": "key",
    "iam_apikey_description": "description",
    "iam_apikey_name": "auto-generated-apikey",
    "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
    "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::",
    "instance_id": "instance_id",
    "url": "https://us-south.ml.cloud.ibm.com",
}

In [ ]:
# The code was removed by Watson Studio for sharing.

このチュートリアルでは、PostgreSQL用のデータベース、Db2 Warehouse、またはPostgreSQLの無料の内部バージョンを使用して、OpenScale用のデータマートを作成することができます。

以前にOpenScaleを設定したことがある場合、既存のデータマートを使用し、現在監視しているモデルに影響ありません。以下のセルは更新しないでください。

有料のクラウドアカウントを持っていない場合、またはこの有料サービスを利用したくない場合は、OpenScaleで無料の内部PostgreSQLサービスを使用することができます。以下のセルは更新しないでください。

Db2 Warehouseの新しいインスタンスをプロビジョニングするには、[クラウドのカタログからDb2 Warehouse](https://cloud.ibm.com/catalog/services/db2-warehouse)を探し、サービス名を指定して、**作成**をクリックします。インスタンスが作成されたら、画面の左側にある **サービス資格情報** リンクをクリックします。**新規資格情報**ボタンをクリックし、資格情報に名前を付けて、**追加**をクリックします。新しい資格情報は、**資格情報の表示**ボタンをクリックしてアクセスできます。Db2 Warehouseの資格情報をコピーして下のセルに貼り付けます。

Databases for PostgreSQL の新しいインスタンスをプロビジョニングするには、[クラウドのカタログからDatabases for PostgreSQL](https://cloud.ibm.com/catalog/services/databases-for-postgresql)を探し、サービス名を指定して **作成** をクリックします。インスタンスが作成されたら、画面左側の **サービス資格情報** リンクをクリックします。**新規資格情報**ボタンをクリックし、資格情報に名前を付けて、**追加**をクリックします。新しい資格情報は、**資格情報の表示**ボタンをクリックしてアクセスできます。PostgreSQL用のデータベースの資格情報をコピーして、以下のセルに貼り付けます。

In [ ]:
DB_CREDENTIALS = None

__以前にOpenScaleで無料の内部バージョンのPostgreSQLを使用するように設定した場合は、有料のデータベースサービスを使用して新しいデータマートに切り替えることができます。__内部のPostgreSQLの設定を削除して、上のセルで提供されたサービスの資格情報を使用して新しいものを作成したい場合は、以下の__KEEP_MY_INTERNAL_POSTGRES__変数を__False__に設定してください。この場合、ノートブックは既存の内部PostgreSQLデータマートを削除し、提供された資格情報を使用して新しいデータマートを作成します。__*データの移行は発生しません。*__

In [ ]:
KEEP_MY_INTERNAL_POSTGRES = True

## notebookを実行する

この時点で、notebookを実行する準備が整いました。セルを 1 つずつ実行するか、上の **Kernel** オプションをクリックして **Restart and Run All** を選択してすべてのセルを実行することもできます。

# モデルの構築とデプロイ<a name="model"></a>

このセクションでは、Spark MLLibモデルのトレーニング方法を学び、次にWatson Machine Learningサービスを利用してWebサービスとしてデプロイする方法を学びます。

## 学習データをgithubからロード

In [ ]:
from IPython.utils import io

with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv -O german_credit_data_biased_training.csv
!ls -lh german_credit_data_biased_training.csv

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import json
import datetime

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

## データを調べる

In [ ]:
df_data.printSchema()

In [ ]:
print("Number of records: " + str(df_data.count()))

## モデルの作成

In [ ]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Spark German Risk Model - Final"
DEPLOYMENT_NAME = "Spark German Risk Deployment - Final"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

 以下のコードは、Sparkでランダムフォレスト分類機(Random Forest Classifier)を作成し、カテゴリ値として利用する特徴量とラベル列に文字列インデクサーを設定しています。最後に、このノートブックでは、インデクサーとモデルを含むパイプラインを作成し、モデルの初期ROC曲線下面積(=AUC Area Under the Curve)評価を行っています。

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.feature import SQLTransformer

features = [x for x in spark_df.columns if x != 'Risk']
categorical_features = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']
categorical_num_features = [x + '_IX' for x in categorical_features]
si_list = [StringIndexer(inputCol=x, outputCol=y) for x, y in zip(categorical_features, categorical_num_features)]
va_features = VectorAssembler(inputCols=categorical_num_features + [x for x in features if x not in categorical_features], outputCol="features")

In [ ]:
si_label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_label.labels)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features")
feature_filter = SQLTransformer(statement="SELECT * FROM __THIS__")
pipeline = Pipeline(stages= si_list + [si_label, va_features, classifier, label_converter, feature_filter])
model = pipeline.fit(train_data)

**注:**　モデル出力からfeatureをフィルタリングしたい場合は、**`*`**　を　**`SQLTransformer`**ステートメントで保持するfeature　nameに置き換えてください。

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

print("areaUnderROC = %g" % area_under_curve)

## モデルの公開

このセクションでは、ノートブックは提供された Watson Machine Learning の資格情報を使用して、モデル（パイプラインを含む）を WML インスタンスに保存します。以前のバージョンのモデルは削除され、ノートブックを再度実行して、別のデモのためにすべてのデータをリセットできるようになります。

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### 既存のモデルとDeploymentyの削除

In [ ]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/credit-risk/meta/credit-risk-meta.json -O credit-risk-meta.json
!ls -lh credit-risk-meta.json

In [ ]:
with open('credit-risk-meta.json') as f:
    [training_data_reference, *_] = json.load(f)['model_meta']['training_data_reference']

In [ ]:
print(training_data_reference)

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": area_under_curve,
           "threshold": 0.7
        }
    ]
}

In [ ]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

In [ ]:
model_uid

## モデルのデプロイ

このノートブックの次のセクションでは、モデルを Watson Machine Learning の RESTful Web サービスとしてデプロイします。デプロイされたモデルには、予測のためにデータをモデルに送信するために使用できるスコアリング URL が用意されています。

In [ ]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# OpenScaleの構成 <a name="openscale"></a>

ここでnotebookに必要なライブラリをインポートしてPython OpenScaleクライアントをセットアップします。

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

### Watson OpenScale GUIDの取得

OpenScaleの各インスタンスには固有のIDがあります。この値は、ノートの冒頭で指定したCloud APIキーを使って取得できます。

In [ ]:
import requests
from ibm_ai_openscale.utils import get_instance_guid


WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

Watson OpenScaleのGUIDが見つからなかった場合は、GUIDを変数 `WOS_GUID` に指定する必要があります。


#### OpenScale `instance_guid` の取得

IBM CLOUD CLIのインストール方法: [インストラクション](https://cloud.ibm.com/docs/cli/reference/ibmcloud?topic=cloud-cli-install-ibmcloud-cli#install_use)

OpenScale インスタンスの GUID を取得する方法

- IBM Cloudにログイン:
```
ibmcloud login
```
- リソースグループが `default` と異なる場合は、OpenScale インスタンスを含むリソースグループに切り替えます。
```
ibmcloud target -g <myResourceGroup>
```
- インスタンスの詳細を取得します
```
ibmcloud resource service-instance 'AI-OpenScale-instance_name'
```

In [ ]:
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
ai_client.version

## スキーマとデータマートの作成

### データマートのセットアップ

Watson OpenScale は、ペイロード・ログと計算されたメトリクスを保存するためにデータベースを使用します。上記でデータベースの資格情報が提供されて**いない**場合、notebookは無料の内部ライトデータベースを使用します。データベースの認証情報が指定された場合、既存のデータマートが**存在しない** **かつ** **KEEP_MY_INTERNAL_POSTGRES** 変数が**True**に設定されていない限り、データマートはそこに作成されます。OpenScaleデータマートがDb2またはPostgreSQLに存在する場合、既存のデータマートが使用され、データは上書きされません。

German Credit モデルの以前のインスタンスは、OpenScaleのモニタリングから削除されます。

In [ ]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        try:
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
        except:
            print('Setup failed, trying Db2 setup')
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS, schema=DB_CREDENTIALS['username'])
    

In [ ]:
data_mart_details = ai_client.data_mart.get_details()

## 機械学習エンジンのバインド

モデルの入出力をペイロードデータを取り込むためには、Watson OpenScale を Watson Machine Learning インスタンスにバインドする必要があります。このバインディングが既に存在する場合、このコードは警告メッセージを出力し、既存のバインディングを使用します。

**注意:**必要に応じて `ai_client.data_mart.binding.add` メソッドを呼び出すことで、複数のエンジンインスタンスをバインドすることができます。そして、`binding_uid` を用いて特定のバインディングを参照することができます。

In [ ]:
binding_uid = ai_client.data_mart.bindings.add('WML Instance 4 Tutorial', WatsonMachineLearningInstance(WML_CREDENTIALS))
bindings_details = ai_client.data_mart.bindings.get_details()

if binding_uid is None:
    binding_uid = [binding['metadata']['guid'] for binding in bindings_details['service_bindings'] if binding['entity']['name']=='WML Cloud Instance'][0]

ai_client.data_mart.bindings.list()

In [ ]:
print(binding_uid)

In [ ]:
ai_client.data_mart.bindings.list_assets(binding_uid=binding_uid)

## サブスクリプション(登録)

### 既存のCreditリスクのサブスクリプションの削除

このコードは、 German Credit モデルの以前のサブスクリプションを削除して、新しいモデルと新しいデータでモニターをリフレッシュします。

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

このコードは、PythonクライアントAPIを使用してOpenScaleでモデルのサブスクリプションを作成します。モデル固有の識別子と、モデル自体に関する情報を提供する必要があることに注意してください。

In [ ]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='predictedLabel',
    probability_column='probability',
    transaction_id_column='transaction_id',
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

サブスクリプション一覧の取得

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

In [ ]:
subscription_details = subscription.get_details()

In [ ]:
subscription.uid

### モデルをスコア化してモニターを設定できるようにする

WMLサービスがバインドされ、サブスクリプションが作成されたので、OpenScaleを設定する前にモデルにリクエストを送信する必要があります。これにより、OpenScale は正しいスキーマでデータマートにペイロードログを作成し、モデルの入出力データをキャプチャすることができます。まず、コードはモデルのデプロイメントのエンドポイントURLを取得し、予測のためにいくつかのレコードを送信します。

In [ ]:
credit_risk_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        credit_risk_scoring_endpoint = deployment['entity']['scoring_url']
        
print(credit_risk_scoring_endpoint)

In [ ]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)

print('Single record scoring result:', '\n fields:', scoring_response['fields'], '\n values: ', scoring_response['values'][0])

In [ ]:
time.sleep(10)
subscription.payload_logging.get_records_count()

# モデル性能モニタリングとフィードバックロギング <a name="quality"></a>

##  モデル性能モニタリングを有効にする

以下のコードは、モニターを有効にする前にペイロードロギングテーブルを設定できるようにするために10秒待ちます。まず、モデル性能（正確度）モニタをオンにして、アラートのしきい値を70%に設定します。OpenScaleは，モデルの正確度測定値（バイナリ分類器の場合は曲線下面積（area under the curve、AUC））がこの閾値を下回ると，ダッシュボードにアラートを表示します．

2番目のパラメータであるmin_recordsは，OpenScaleが新しい測定値を計算する前に必要なフィードバックレコードの最小数を指定します．モデル性能モニタは1時間ごとに実行されますが，ユーザーインタフェース，Pythonクライアント，または提供されたフィードバックエンドポイントを介して，50個のフィードバックレコードが追加されるまで，ダッシュボードの正確度表示は変化しません．

In [ ]:
time.sleep(10)
subscription.quality_monitoring.enable(threshold=0.7, min_records=50)

## フィードバックロギング

以下のコードは、OpenScaleが新しい正確度測定値を計算できるように、最小のしきい値を満たすのに十分なフィードバック・データをダウンロードして保存します。そして、モデル性能モニタを起動します。モニタは1時間ごとに実行されるか、Python API、REST API、またはグラフィカル・ユーザ・インタフェースを介して起動することができます。

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/additional_feedback_data.json -O additional_feedback_data.json
!ls -lh additional_feedback_data.json

In [ ]:
with open('additional_feedback_data.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)
    
subscription.feedback_logging.store(additional_feedback_data['data'])

In [ ]:
subscription.feedback_logging.show_table()

In [ ]:
run_details = subscription.quality_monitoring.run(background_mode=False)

In [ ]:
time.sleep(10)
subscription.quality_monitoring.show_table()

In [ ]:
%matplotlib inline

quality_pd = subscription.quality_monitoring.get_table_content(format='pandas')
quality_pd.plot.barh(x='id', y='value');

# 公平性、ドリフトモニタリング と 説明 <a name="fairness"></a>

### 公平性の設定

以下のコードは、モデルの公平性モニタリングを設定しています。これは、Sex(性別)とAge(年齢)の2つの機能の監視をオンにします。それぞれのケースで、指定する必要があります:

  * モデルのどの属性(特徴量)を監視するか
  * 1つまたは複数の **majority** グループ、好ましい結果の割合が高くなると予想されるその属性の値
  * 1つまたは複数の **minority** グループ、好ましくない結果の割合が高いと予想される属性の値
  * 公平性の測定値が(この場合は95%)を下回った場合に、OpenScaleにアラートを表示させる閾値
  
 さらに、モデルのどの結果が好ましい結果で、どの結果が好ましくないかを指定する必要があります。また、OpenScaleが公平性値を計算するために使用するレコードの数も指定しなければなりません。この場合、OpenScaleの公平性モニターは1時間ごとに実行されますが、少なくとも200レコードが追加されるまでは、新しい公平性レーティングは計算されません。最後に、公平性を計算するために、OpenScaleはトレーニングデータに対していくつかの計算を行う必要があるので、そのデータを含むデータフレームを提供します。

In [ ]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.95),
                Feature("Age", majority=[[26,75]], minority=[[18,25]], threshold=0.95)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=200,
            training_data=pd_data
        )

### ドリフトの設定

In [ ]:
subscription.drift_monitoring.enable(threshold=0.05, min_records=200)

In [ ]:
drift_status = None
while drift_status != 'finished':
    drift_details = subscription.drift_monitoring.get_details()
    drift_status = drift_details['parameters']['config_status']['state']
    if drift_status != 'finished':
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)
print(drift_status)

## モニタリングが設定されたので、モデルを再度評価します

次のセクションでは、データフィードから200レコードをランダムに選択し、それらのレコードを予測のためにモデルに送ります。これは、前のセクションで設定したレコードの最小閾値を超えるのに十分な量であり、これによりOpenScaleは公平性の計算を開始することができます。

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_feed.json -O german_credit_feed.json
!ls -lh german_credit_feed.json

ランダムに選ばれた200行ののレコードを予測

In [ ]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}


scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)
time.sleep(5)

**注：**ペイロードテーブルには、合計208レコードがあるはずです。

In [ ]:
print('Number of records in payload table: ', subscription.payload_logging.get_records_count())

## 公平性モニターの起動

現在のデータで公平性モニターの実行をキックします。モニターは1時間ごとに実行されますが、Pythonクライアント、REST API、グラフィカルユーザーインターフェースを使用して手動で起動することもできます。

In [ ]:
fairness_run_details = subscription.fairness_monitoring.run(background_mode=False)

In [ ]:
time.sleep(10)

subscription.fairness_monitoring.show_table()

## ドリフトモニターの起動

現在のデータでドリフトモニターの実行をキックします。モニターは1時間ごとに実行されますが、Pythonクライアント、REST APIを使用して手動で開始することもできます。

In [ ]:
drift_run_details = subscription.drift_monitoring.run(background_mode=False)

In [ ]:
subscription.drift_monitoring.get_table_content()

**注:** 以下のセルでドリフトモニタの実行コードを無効にすることができます。

In [ ]:
subscription.drift_monitoring.disable()

## 説明性の設定

最後に、OpenScaleに訓練データを提供し、説明性の機能を有効化して設定します。

In [ ]:
from ibm_ai_openscale.supporting_classes import *

subscription.explainability.enable(training_data=pd_data)

In [ ]:
explainability_details = subscription.explainability.get_details()

## サンプルレコードに対して説明性データの作成

In [ ]:
transaction_id = subscription.payload_logging.get_table_content(limit=1)['scoring_id'].values[0]

print(transaction_id)

In [ ]:
explain_run = subscription.explainability.run(transaction_id=transaction_id, background_mode=False, cem=False)

In [ ]:
explain_result = pd.DataFrame.from_dict(explain_run['entity']['predictions'][0]['explanation_features'])
explain_result.plot.barh(x='feature_name', y='weight', color='g', alpha=0.8);

# カスタム・モニターと指標の作成 <a name="custom"></a>

## カスタム・モニターの登録

In [ ]:
def get_definition(monitor_name):
    monitors_definitions = ai_client.data_mart.monitors.get_details()['monitor_definitions']
    
    for definition in monitors_definitions:
        if monitor_name == definition['entity']['name']:
            return definition
    
    return None

In [ ]:
from ibm_ai_openscale.supporting_classes import Metric, Tag

monitor_name = 'my model performance'
metrics = [Metric(name='sensitivity', lower_limit_default=0.8), Metric(name='specificity', lower_limit_default=0.75)]
tags = [Tag(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    my_monitor = ai_client.data_mart.monitors.add(name=monitor_name, metrics=metrics, tags=tags)
else:
    my_monitor = existing_definition

### モニターの一覧

### モニターのuidと詳細の取得

In [ ]:
monitor_uid = my_monitor['metadata']['guid']

print(monitor_uid)

In [ ]:
my_monitor = ai_client.data_mart.monitors.get_details(monitor_uid=monitor_uid)
print('monitor definition details', my_monitor)

## サブスクリプションに対してカスタム・モニターを有効化する

In [ ]:
from ibm_ai_openscale.supporting_classes import Threshold

thresholds = [Threshold(metric_uid='sensitivity', lower_limit=0.9)]
subscription.monitoring.enable(monitor_uid=monitor_uid, thresholds=thresholds)

### モニター設定の詳細の取得

In [ ]:
subscription.monitoring.get_details(monitor_uid=monitor_uid)

## カスタム指標の保存

In [ ]:
metrics = {"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}

subscription.monitoring.store_metrics(monitor_uid=monitor_uid, metrics=metrics)

### カスタム指標の一覧と取得

In [ ]:
time.sleep(5)

subscription.monitoring.show_table(monitor_uid=monitor_uid)

In [ ]:
custom_metrics = subscription.monitoring.get_metrics(monitor_uid=monitor_uid, deployment_uid='credit')
custom_metrics

In [ ]:
custom_metrics_pandas = subscription.monitoring.get_table_content(monitor_uid=monitor_uid)

%matplotlib inline
custom_metrics_pandas.plot.barh(x='id', y='value');

# ペイロード分析 <a name="analytics"></a>

## データ分布の計算を実行

In [ ]:
from datetime import datetime

start_date = "2018-01-01T00:00:00.00Z"
end_date = datetime.utcnow().isoformat() + "Z"

sex_distribution = subscription.payload_logging.data_distribution.run(
            start_date=start_date,
            end_date=end_date,
            group=['predictedLabel', 'Sex'],
            agg=['count'])

## pandasのdataframeとしてデータ分布を取得する

In [ ]:
sex_distribution_run_uid = sex_distribution['id']
distributions_pd = subscription.payload_logging.data_distribution.get_run_result(run_id=sex_distribution_run_uid, format='pandas')
distributions_pd

In [ ]:
subscription.payload_logging.data_distribution.show_chart(sex_distribution_run_uid);

In [ ]:
credit_history_distribution = subscription.payload_logging.data_distribution.run(
            start_date=start_date,
            end_date=end_date,
            group=['predictedLabel', 'CreditHistory'],
            agg=['count'])

In [ ]:
credit_history_distribution_run_uid = credit_history_distribution['id']

subscription.payload_logging.data_distribution.show_chart(credit_history_distribution_run_uid);

# ビジネス アプリケーション<a name="application"></a>

In [ ]:
from ibm_ai_openscale.supporting_classes import BusinessMetric, AggregationMethods

次のセルでは、アプリケーション用に `business payload fields` と `business metrics` を記述します`business payload fields`は、"履歴データ"セクションの最後に読み込まれるビジネスペイロードのカラムのリストです。`business metrics`は、作成したアプリケーションで計算される指標(BKPIs)を定義します。アプリケーションは履歴データを読み込んだ後、[ビジネスアプリケーションの実行　のセクション](#run_application)で実行されます。

In [ ]:
business_payload_fields = [
                {
                    "name": "LoanDuration",
                    "type": "number",
                    "description": "Duration of the loan"
                },
                {
                    "name": "LoanPurpose",
                    "type": "string",
                    "description": "Purpose of the loan"
                },
                {
                    "name": "LoanAmount",
                    "type": "number",
                    "description": "Amount of the loan"
                },
                {
                    "name": "InstallmentPercent",
                    "type": "number",
                    "description": "Installment percents"
                },
                {
                    "name": "AcceptedPercent",
                    "type": "number"
                },
                {
                    "name": "AmountGranted",
                    "type": "number",
                    "description": "Risk percent"
                },
                {
                    "name": "Accepted",
                    "type": "number",
                    "description": "Number of loans accepted"
                }
            ]

In [ ]:
business_metrics = [BusinessMetric(metric_name='Accepted Credits', description="Accepted Credits Daily",
                                           field_name='Accepted', aggregation=AggregationMethods.SUM,
                                   time_unit='day', time_count=1, lower_limit=2150),
                    BusinessMetric(metric_name='Credit Amount Granted', description="Credit Amount Granted Daily",
                                   field_name='AmountGranted', aggregation=AggregationMethods.SUM,
                                   time_unit='day', time_count=1, lower_limit=5500000)]

## ビジネス アプリケーションの追加と詳細の取得:

In [ ]:
application_details = ai_client.data_mart.applications.add(name="Tutorial Credit Risk Application",
                                                           description="Tutorial Business Application",
                                                           subscription_ids=[subscription.uid],
                                                           business_metrics=business_metrics,
                                                           payload_schema=business_payload_fields)
application_id = application_details['metadata']['id']

In [ ]:
while application_details['entity']['status']['state'] == 'preparing':
    print(datetime.utcnow().strftime('%H:%M:%S'), application_details['entity']['status']['state'] )
    time.sleep(10)
    application_details = ai_client.data_mart.applications.get_details(application_id)
print("Final application status: {}".format(application_details['entity']['status']['state']) )

Uncomennt cell to show details:

application_details

# 履歴データ <a name="historical"></a>

In [ ]:
historyDays = 7

## ペイロード履歴の挿入

notebookの次のセクションでは、過去7日間監視され定期的にトラフィックを受信している本番モデルをシミュレーショ ンするために、ペイロードと測定テーブルに履歴データをダウンロードして書き込みます。この履歴データは、Watson OpenScale のユーザーインターフェースで確認できます。このコードでは、Python と REST API を使用してこのデータを書き込みます。

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_0.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_1.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_2.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_3.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_4.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_5.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_6.json
!ls -lh history_payloads_with_transaction_id_*.json

In [ ]:
from ibm_ai_openscale.utils.inject_demo_data import DemoData
import os

historicalData = DemoData(aios_credentials=WOS_CREDENTIALS)
historical_data_path=os.getcwd()

historicalData.load_historical_scoring_payload(subscription, deployment_uid,file_path=historical_data_path, day_template="history_payloads_with_transaction_id_{}.json" )

In [ ]:
performance_metrics_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

## 公平性指標の履歴の挿入

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_fairness.json -O history_fairness.json
!ls -lh history_fairness.json

In [ ]:
with open('history_fairness.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    metrics = []
    
    for hour in range(24):
        score_time = (datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed
        
        metric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': payloads[index]
        }
        metrics.append(metric)
    response = requests.post(performance_metrics_url, json=metrics, headers=ai_client._get_headers())
print('Finished')

## バイアス緩和指標履歴の挿入

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_debias.json -O history_debias.json
!ls -lh history_debias.json

In [ ]:
with open('history_debias.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    debias_metrics = []
    for hour in range(24):
        score_time = (datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed

        debiasMetric = {
            'metric_type': 'debiased_fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': subscription.uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': payloads[index]
        }

        debias_metrics.append(debiasMetric)
    response = requests.post(performance_metrics_url, json=debias_metrics, headers=ai_client._get_headers())
print('Finished')

## モデル性能指標履歴の挿入

In [ ]:
measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    quality_metrics = []
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': subscription.uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.7,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.7
                    }
                ]
            }
        }
        
        quality_metrics.append(qualityMetric)
    
    response = requests.post(performance_metrics_url, json=quality_metrics, headers=ai_client._get_headers())

print('Finished')

## 混同行列(confusion matrixes)履歴の挿入

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_quality_metrics.json -O history_quality_metrics.json
!ls -lh history_quality_metrics.json

In [ ]:
measurements_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/measurements'
print(measurements_url)

In [ ]:
with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)
    
for day in range(historyDays):
    index = 0
    measurments = []
    print('Day', day + 1)
    
    for hour in range(24):
        score_time = (datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')

        measurement = {
            "monitor_definition_id": 'quality',
            "binding_id": subscription.binding_uid,
            "subscription_id": subscription.uid,
            "asset_id": subscription.source_uid,
            'metrics': [records[index]['metrics']],
            'sources': [records[index]['sources']],
            'timestamp': score_time
        }

        measurments.append(measurement)
        index+=1

    response = requests.post(measurements_url, json=measurments, headers=ai_client._get_headers())

print('Finished')

## パフォーマンス指標履歴の挿入

In [ ]:
for day in range(historyDays):
    performance_metrics = []
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(60, 600)
        score_resp = random.uniform(60, 300)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': subscription.uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }
        performance_metrics.append(performanceMetric)

    response = requests.post(performance_metrics_url, json=performance_metrics, headers=ai_client._get_headers())

print('Finished')

## マニュアルラベリング履歴の挿入

In [ ]:
manual_labeling_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/manual_labelings'
print(manual_labeling_url)

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_manual_labeling.json -O history_manual_labeling.json
!ls -lh history_manual_labeling.json

In [ ]:
with open('history_manual_labeling.json', 'r') as history_file:
    records = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    record_json = []
    for hour in range(24):
        for record in records:
            if record['fastpath_history_day'] == day and record['fastpath_history_hour'] == hour:
                record['binding_id'] = binding_uid
                record['subscription_id'] = subscription.uid
                record['asset_revision'] = model_uid
                record['deployment_id'] = deployment_uid
                record['scoring_timestamp'] = (datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
                record_json.append(record)
    response = requests.post(manual_labeling_url, json=record_json, headers=ai_client._get_headers())

print('Finished')

## ドリフト測定値履歴の挿入

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_0.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_1.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_2.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_3.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_4.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_5.json
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_6.json
!ls -lh history_drift_measurement_*.json

In [ ]:
print(measurements_url)

In [ ]:
for day in range(historyDays):
    measurements = []

    with open("history_drift_measurement_{}.json".format(day), 'r') as history_file:
        drift_daily_measurements = json.load(history_file)
    print('Loading day', day + 1)

    #Historical data contains 8 records per day - each represents 3 hour drift window.
    
    for nb_window, records in enumerate(drift_daily_measurements):
        for record in records:
            window_start =  (datetime.utcnow() + timedelta(hours=(-(24 * day + (nb_window+1)*3 + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ') # first_payload_record_timestamp_in_window (oldest)
            window_end = (datetime.utcnow() + timedelta(hours=(-(24 * day + nb_window*3 + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')# last_payload_record_timestamp_in_window (most recent)
            #modify start and end time for each record
            record['sources'][0]['data']['start'] = window_start
            record['sources'][0]['data']['end'] = window_end
            
            measurement = {
                "monitor_definition_id": "drift",
                "binding_id": subscription.binding_uid,
                "subscription_id": subscription.uid,
                "asset_id": subscription.source_uid,
                'metrics': record['metrics'],
                'sources': record['sources'],
                'timestamp': window_start,
                'process': 'Drift run for subscription_{}'.format(subscription.uid)
            }

            measurements.append(measurement)
    response = requests.post(measurements_url, json=measurements, headers=ai_client._get_headers())
    print("Daily loading finished.")

## ビジネスペーロード履歴の挿入

以下のセクションでは、すべての履歴レコードが正常にロードされるまでの待ち時間を2分間をトリガーとしています。

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_business_payloads_week.csv -O history_business_payloads_week.csv
!ls -lh history_business_payloads_week.csv

In [ ]:
business_payload_data_set_id = application_details['entity']['business_payload_data_set_id']

historicalData.load_historical_business_payload(business_payload_data_set_id, file_path=historical_data_path, file_name="history_business_payloads_week.csv")
time.sleep(120)

# ビジネス アプリケーションの実行  <a id="run_application"></a>

In [ ]:
run_details = ai_client.data_mart.applications.run(application_id)

## デバッグに役立つ追加データ

In [ ]:
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
print('Scoring URL:', credit_risk_scoring_endpoint)

## 説明性表示のためにトランザクションを特定する

以下のセルで特定されるトランザクション ID は、OpenScale ダッシュボードの説明可能性タブにコピーして貼り付けることができます。

In [ ]:
payload_data = subscription.payload_logging.get_table_content(limit=20)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability'])

In [ ]:
# The code was removed by Watson Studio for sharing.

 ## おめでとうございます。

IBM Watson OpenScale のハンズオン・ラボが終了しました。これで [OpenScale Dashboard] (https://aiopenscale.cloud.ibm.com/) を表示できるようになりました。German Credit モデルのタイルをクリックすると、公平性、品質、およびパフォーマンス・モニターが表示されます。時系列グラフをクリックすると、特定の時間ウィンドウ内のトランザクションの詳細情報が表示されます。

## 次のステップ

OpenScaleは、モデルのパフォーマンスを時間の経過とともに表示します。OpenScaleのグラフにデータを流し続けるには、次の2つのオプションがあります。

 *  [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb).をダウンロードして、構成し、スケジュールを設定します。このノートブックは、WML資格情報を使用して設定し、モデルへのスコアリング要求の一貫したフローを提供するようにスケジュールすることができ、OpenScaleモニタに表示されます。
 * このノートブックを再実行します。このノートブックを最初から実行すると、モデルと配置が削除されて再作成され、履歴データが再作成されます。前回の配置のペイロードと測定ログはデータマートに保存され続け、必要に応じて削除することができます。

## 執筆者の紹介

Eric Martensは、ビジネスプロセスの分析と記述、およびそれらの機能・非機能IT要件への変換を専門とする技術スペシャリストです。彼は、ITとビジネスの世界の間の通訳者としての役割を果たしています。

Lukasz Cmielowski（PhD)は、IBMのオートメーション・アーキテクト兼データ・サイエンティストで、データを実用的な知識に変換するクライアントの能力を大幅に向上させるエンタープライズ・レベルのアプリケーション開発の実績があります。